In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pickle
from glob import glob
import re
from concurrent.futures import ProcessPoolExecutor, as_completed

import numpy as np
import pandas as pd
#from tqdm import tqdm
from scipy import stats
from sklearn.metrics import pairwise_distances

import utils.constants as constants
from data.multixcan_data import MXPhenoInfo, MXPhenoResults
from utils.utils import is_number, chunker

In [3]:
genes_associations_dir = os.path.join(constants.PREPROCESSED_BASED_DIR, 'gene_associations')

In [4]:
OUTPUT_DIR = os.path.join(genes_associations_dir, 'fastenloc')
display(OUTPUT_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

'/mnt/phenomexcan/results/preprocessed_data/gene_associations/fastenloc'

In [5]:
FASTENLOC_RESULTS_DIR = os.path.join(constants.RESULTS_DIR, 'fastenloc', 'torus_based')
display(FASTENLOC_RESULTS_DIR)

'/mnt/phenomexcan/results/fastenloc/torus_based'

In [6]:
N_EXPECTED_PHENOTYPES = 4083
N_EXPECTED_TISSUES = 49

# Load all S-PrediXcan gene mappings

In [7]:
with open(os.path.join(constants.PREPROCESSED_METADATA_DIR, 'genes_mapping_simplified-0.pkl'), 'rb') as f:
    genes_mapping_0 = pickle.load(f)

with open(os.path.join(constants.PREPROCESSED_METADATA_DIR, 'genes_mapping_simplified-1.pkl'), 'rb') as f:
    genes_mapping_1 = pickle.load(f)

In [8]:
all_gene_ids = set(genes_mapping_0.keys())

# fastENLOC reading functions

In [9]:
# create summary of files
all_fastenloc_results_files = glob(os.path.join(FASTENLOC_RESULTS_DIR, '*.enloc.sig.out.gz'))
all_fastenloc_phenotypes = [f.split('-')[1] for f in all_fastenloc_results_files]
all_fastenloc_tissues = [f.split('.enloc.sig.out.gz')[0].split('-')[-1] for f in all_fastenloc_results_files]

results_summary = pd.DataFrame({'pheno': all_fastenloc_phenotypes, 'tissue': all_fastenloc_tissues, 'file': all_fastenloc_results_files})

In [10]:
assert len(set(all_fastenloc_phenotypes)) == N_EXPECTED_PHENOTYPES
assert len(set(all_fastenloc_tissues)) == N_EXPECTED_TISSUES
assert len(all_fastenloc_results_files) == int(N_EXPECTED_PHENOTYPES * N_EXPECTED_TISSUES)

In [11]:
def read_fe(filename, rename_genes=True): 
    fe_data = pd.read_csv(filename, sep='\s+', usecols=[0, 5], header=None).rename(columns={0: 'gene_cluster', 5: 'rcp'})
    fe_data = fe_data = fe_data.assign(gene=fe_data['gene_cluster'].apply(lambda x: x.split(':')[0])) 
    fe_data = fe_data.assign(gene_cluster=fe_data['gene_cluster'].apply(lambda x: x.split(':')[1]))
    
    # FIXME: is it correct to sum across clusters of a gene? or pick max? Haky thinks summing is fine.
    fe_max = fe_data.groupby('gene')['rcp'].sum()
    fe_max = fe_max.loc[fe_max.index.intersection(all_gene_ids)]
    if rename_genes:
        fe_max = fe_max.rename(index=genes_mapping_0)
    return fe_max

def read_pheno(pheno): 
    _tmp = results_summary[results_summary['pheno'] == pheno] 
    if _tmp.shape[0] == 0: 
        return None 
     
    _res = {} 
    _all_genes = set() 
    for idx, _data in _tmp.iterrows(): 
        #print(_data.tissue) 
         
        tissue_data = read_fe(_data.file) 
        _res[_data.tissue] = tissue_data 
        _all_genes.update(tissue_data.index) 
    
    return pd.DataFrame(_res, index=_all_genes).fillna(-1).max(axis=1)

In [12]:
# testing
t = read_fe(os.path.join(constants.RESULTS_DIR, 'fastenloc', 'torus_based', 'fastenloc-J15-Whole_Blood.enloc.sig.out.gz'))

In [13]:
genes_mapping_1['SLCO4C1']

'ENSG00000173930'

In [14]:
genes_mapping_0['ENSG00000238009']

'RP11-34P13.7'

In [15]:
t.sort_values(ascending=False).head()

SLCO4C1    1.869481e-07
ENPP4      2.257609e-08
EIF3F      1.147284e-08
CREM       1.070100e-08
RNF144A    4.772313e-09
Name: rcp, dtype: float64

In [16]:
assert t['SLCO4C1'] == (1.869e-07 + 1.578e-11 + 5.201e-12 + 1.409e-13 + 2.694e-11)

In [17]:
assert t['RP11-34P13.7'] == 0.00, t['RP11-34P13.7']

```parallel -j4 'zcat {} | grep ENSG00000049246' ::: fastenloc-1180-*.gz | column -t | sort -k6 -g```

In [18]:
display(all_fastenloc_phenotypes[0])
t = read_pheno(all_fastenloc_phenotypes[0])

'J15'

In [19]:
t.sort_values(ascending=False).head()

BMP8B      0.011245
OXCT2      0.009545
SLCO4C1    0.000965
ENPP4      0.000835
ENPP5      0.000821
dtype: float64

In [20]:
genes_mapping_1['BMP8B']

'ENSG00000116985'

In [21]:
t.loc['BMP8B']

0.01124506203

In [22]:
assert t.loc['BMP8B'] == (1.124e-02 + 5.425e-07 + 1.153e-08 + 2.864e-06 + 1.644e-06)

# Get all S-MultiXcan phenos

In [23]:
all_smultixcan_files = glob(os.path.join(constants.SMULTIXCAN_MASHR_RESULTS_DIR, '*.tsv.gz'))
all_smultixcan_phenotypes = [MXPhenoResults(p) for p in all_smultixcan_files]

In [24]:
assert len(all_smultixcan_files) == len(all_smultixcan_phenotypes) == N_EXPECTED_PHENOTYPES

## Compute results

In [25]:
def _get_combined_results(phenos):
    return {pheno.pheno_info.get_plain_name() : read_pheno(pheno.pheno_info.pheno_code) for pheno in phenos}

In [ ]:
# testing
_tmp = _get_combined_results(all_smultixcan_phenotypes[:4])
assert len(_tmp) == 4

In [40]:
_pending = read_pheno('C_MYELOID')

In [41]:
_pending.head()

DFNA5       7.883540e-10
RPS27       4.784000e-12
KANSL1      2.464491e-10
BROX        4.186500e-11
ARHGEF18    3.673000e-10
dtype: float64

### pvalues and z-scores

In [27]:
output_file = os.path.join(OUTPUT_DIR, f'fastenloc-rcp.pkl.xz')
display(output_file)

'/mnt/phenomexcan/results/preprocessed_data/gene_associations/fastenloc/fastenloc-rcp.pkl.xz'

In [28]:
def _run_all(phenotype_chunks, n_jobs=20):
    all_results = {}
    
    with ProcessPoolExecutor(max_workers=n_jobs) as executor:
        tasks = [executor.submit(_get_combined_results, chunk) for chunk in phenotype_chunks]
        for future in as_completed(tasks):
            res = future.result()
            all_results.update(res)
    
    # Save all results from S-PrediXcan
    with open(output_file, 'wb') as f:
        pickle.dump(all_results, f)

In [29]:
# phenotype_chunks = list(chunker(all_smultixcan_phenotypes[:5], 2))
phenotype_chunks = chunker(all_smultixcan_phenotypes, 200)

In [ ]:
_run_all(phenotype_chunks)

## Load results and save as DataFrame

In [30]:
with open(output_file, 'rb') as f:
    all_results = pickle.load(f)

In [46]:
# all_results['C_MYELOID-LEUKAEMIA-Myeloid_leukaemia'] = _pending

In [47]:
# with open(output_file, 'wb') as f:
#     pickle.dump(all_results, f)

In [48]:
assert len(all_results) == N_EXPECTED_PHENOTYPES

In [49]:
spredixcan_genes_associations = pd.DataFrame(all_results)
spredixcan_genes_associations.index.rename('gene_name', inplace=True)

assert spredixcan_genes_associations.index.is_unique

display(spredixcan_genes_associations.shape)
display(spredixcan_genes_associations.head())

(22233, 4083)

,L12_NAILDIS-Nail_disorders,J69-Diagnoses_main_ICD10_J69_Pneumonitis_due_to_solids_and_liquids,R07-Diagnoses_main_ICD10_R07_Pain_in_throat_and_chest,M46-Diagnoses_main_ICD10_M46_Other_inflammatory_spondylopathies,20003_2038460150-Treatmentmedication_code_paracetamol,E10-Diagnoses_main_ICD10_E10_Insulindependent_diabetes_mellitus,5134_raw-6mm_strong_meridian_left,E86-Diagnoses_main_ICD10_E86_Volume_depletion,L12_ERYTHEMATOUSOTH-Other_erythematous_conditions,K83-Diagnoses_main_ICD10_K83_Other_diseases_of_biliary_tract,...,20090_394-Type_of_fatoil_used_in_cooking_Unknown_soft_margarine,22617_3512-Job_SOC_coding_Aircraft_pilots_and_flight_engineers,6034-Target_heart_rate_achieved,20003_1140883066-Treatmentmedication_code_insulin_product,22601_41223241-Job_coding_accounts_and_wages_clerkassistantsupervisor_bookkeeper_cost_or_ledger_clerk_audit_assistant_budget_officer_student_loans_officer_paymaster,I82-Diagnoses_main_ICD10_I82_Other_venous_embolism_and_thrombosis,20107_12-Illnesses_of_father_Severe_depression,B07-Diagnoses_main_ICD10_B07_Viral_warts,22601_12253140-Job_coding_sports_centre_manager_riding_school_owner_sports_ground_manager_baths_manager,2664_2-Reason_for_reducing_amount_of_alcohol_drunk_Doctors_advice
gene_name,,,,,,,,,,,,,,,,,,,,,
A1BG,1.515300e-08,2.216225e-08,0.005937,5.482960e-10,0.006081,0.000458,0.006905,2.241375e-10,1.753100e-09,2.774200e-05,...,0.000292,1.498710e-10,6.837000e-06,0.000714,1.503420e-10,5.317725e-08,5.242997e-04,1.807621e-08,7.545600e-08,0.000154
A1CF,2.796517e-11,1.696862e-10,0.001970,1.021794e-10,0.010776,0.000006,0.005850,7.833300e-11,2.507511e-10,3.895151e-10,...,0.000225,1.869519e-10,4.081060e-11,0.000270,2.783110e-11,1.083806e-10,3.367239e-07,5.118762e-11,1.050704e-10,0.000449
A2M,1.926980e-08,7.858200e-09,0.000947,5.466000e-11,0.003225,0.000025,0.003633,4.745490e-10,5.610300e-10,9.633650e-06,...,0.000081,1.608430e-10,3.141280e-06,0.000401,1.096541e-10,3.252662e-08,5.695800e-04,1.128942e-08,1.132000e-08,0.000130
A2ML1,1.142270e-09,3.492600e-09,0.000428,1.476100e-10,0.004078,0.000247,0.018624,3.055700e-10,3.589500e-10,3.792130e-06,...,0.000197,2.759200e-10,7.272498e-06,0.000148,1.642360e-11,8.912710e-08,8.749040e-04,1.326800e-10,2.235800e-09,0.000005
A3GALT2,1.997480e-08,5.515000e-10,0.002141,2.412720e-09,0.004778,0.000200,0.141350,1.314787e-10,1.597080e-09,2.149724e-05,...,0.000311,2.357997e-10,1.826130e-09,0.000989,6.655500e-11,1.471335e-10,6.330287e-04,4.799799e-09,1.115040e-07,0.000050


In [50]:
spredixcan_genes_associations.isna().sum().sum()

0

In [51]:
# _tmp = spredixcan_genes_associations.dropna(axis=1, how='all').columns

In [52]:
# spredixcan_genes_associations.columns.difference(_tmp)

In [53]:
# Remove genes with no results
spredixcan_genes_associations = spredixcan_genes_associations.dropna(axis=0, how='all')
assert spredixcan_genes_associations.isna().sum().sum() == 0

display(spredixcan_genes_associations.shape)

(22233, 4083)

In [55]:
assert spredixcan_genes_associations.shape == (22233, N_EXPECTED_PHENOTYPES)

In [56]:
# some testing

# For FinnGen
# assert spredixcan_genes_associations.loc['ENSG00000110628', 'C3_TONGUENAS-Malignant_neoplasm_of_other_and_unspecified_parts_of_tongue'] == 0.005086576789507484
# assert spredixcan_genes_associations.loc['ENSG00000169783', 'C3_TONGUENAS-Malignant_neoplasm_of_other_and_unspecified_parts_of_tongue'] == 0.3757187601354043
# assert spredixcan_genes_associations.loc['ENSG00000137959', 'C3_TONGUENAS-Malignant_neoplasm_of_other_and_unspecified_parts_of_tongue'] == 5.132614371931036e-07
# # For ICD10
# assert spredixcan_genes_associations.loc['ENSG00000135775', 'N18-Diagnoses_main_ICD10_N18_Chronic_renal_failure'] == 2.795075036067939e-05
# assert spredixcan_genes_associations.loc['ENSG00000169783', 'N18-Diagnoses_main_ICD10_N18_Chronic_renal_failure'] == 0.06668736815697908
# assert spredixcan_genes_associations.loc['ENSG00000174226', 'N18-Diagnoses_main_ICD10_N18_Chronic_renal_failure'] == 0.02496852053808064

In [57]:
# Save
spredixcan_genes_associations_filename = os.path.join(OUTPUT_DIR, f'fastenloc-torus-genes_rcp.pkl.xz')
display(spredixcan_genes_associations_filename)

'/mnt/phenomexcan/results/preprocessed_data/gene_associations/fastenloc/fastenloc-torus-genes_rcp.pkl.xz'

In [ ]:
spredixcan_genes_associations.to_pickle(spredixcan_genes_associations_filename)

# Export tables for web publication

In [ ]:
import yaml
from clustering.biclustering.analysis import Trait

In [ ]:
spredixcan_genes_associations.head()

In [ ]:
ukb_traits = spredixcan_genes_associations.columns.tolist()

In [ ]:
len(ukb_traits)

In [ ]:
ukb_traits[:5]

In [ ]:
file_template = '{trait_code}'

In [ ]:
filename_to_description = {}

for t in ukb_traits:
    trait = Trait(t)
    trait_code = trait.trait_code
    filename = file_template.format(trait_code=trait_code)
    
    trait_plain_name = trait.trait_plain_name
    if pd.isnull(trait_plain_name):
        trait_plain_name = filename
    
    filename_to_description[filename] = trait_plain_name

In [ ]:
# should be empty
[(k, v) for k, v in filename_to_description.items() if pd.isnull(v)]

In [ ]:
len(filename_to_description)

In [ ]:
with open('/mnt/tmp/ukb_filename_maps.yml', 'w') as outfile:
    yaml.dump({'ukb_name': filename_to_description}, outfile, default_flow_style=False, default_style='yaml_dump')

Now I need to copy this file to CRI to include only these genes